In [ ]:
import ccxt
import pandas as pd
import os

# --- Configurable Parameters ---
TRADE_AMOUNT = 10  # Trade amount in USD
DEFAULT_FEE = 0.001  # Default maker/taker fee (0.1% per trade)
RESULTS_FILE = "arbitrage_results_2.csv"

# --- Function to Fetch Liquidity ---
def get_liquidity(exchange_name, symbol):
    """Fetches the 24-hour trading volume (liquidity) for a given symbol on Binance or Bybit."""
    try:
        exchange = getattr(ccxt, exchange_name)()

        # Try fetching liquidity from fetch_ticker
        ticker = exchange.fetch_ticker(symbol)
        quote_volume = ticker.get("quoteVolume", None) or ticker.get("baseVolume", None)

        # If ticker volume is missing, check order book liquidity
        if not quote_volume:
            order_book = exchange.fetch_order_book(symbol)
            top_bid = order_book["bids"][0][1] if order_book["bids"] else 0
            top_ask = order_book["asks"][0][1] if order_book["asks"] else 0
            quote_volume = top_bid + top_ask  # Sum of top bid & ask volumes

        return quote_volume if quote_volume else "N/A"

    except Exception as e:
        print(f"⚠️ Error fetching liquidity for {symbol} on {exchange_name}: {e}")
        return "N/A"

# --- Function to Get Exchange Info ---
def get_exchange_info(exchange_name):
    """Fetches trading information for a given exchange, including fees and liquidity data."""
    try:
        exchange = getattr(ccxt, exchange_name)()

        # Always use default fee values (0.1% per trade)
        trading_fees = {"maker_fee": DEFAULT_FEE, "taker_fee": DEFAULT_FEE}

        # Fetch market symbols (USDT pairs only)
        markets = exchange.load_markets()
        usdt_pairs = [s for s in markets.keys() if "USDT" in s]

        return {
            "Exchange": exchange_name,
            "Trading Fees": trading_fees,
            "Supported Symbols": usdt_pairs  # List of tradable USDT pairs
        }
    
    except Exception as e:
        print(f"⚠️ Could not retrieve data for {exchange_name}: {e}")
        return {"Error": str(e)}

# --- Function to Compare Fees, Spreads, and Liquidity ---
def compare_fees_spreads_liquidity(exchange1_name, exchange2_name):
    """Compares trading fees, spreads, and liquidity for common USDT pairs between two exchanges."""
    if exchange1_name == exchange2_name:
        raise ValueError("❌ Both exchanges are the same. Please enter two different exchanges.")

    if not hasattr(ccxt, exchange1_name) or not hasattr(ccxt, exchange2_name):
        raise ValueError("❌ One or both exchanges are not listed in CCXT. Check your inputs.")

    # Fetch exchange information
    exchange1_info = get_exchange_info(exchange1_name)
    exchange2_info = get_exchange_info(exchange2_name)

    # Find common trading symbols
    symbols1 = set(exchange1_info.get("Supported Symbols", []))
    symbols2 = set(exchange2_info.get("Supported Symbols", []))
    common_symbols = list(symbols1 & symbols2)

    print(f"✅ Found {len(common_symbols)} common USDT trading pairs between {exchange1_name} and {exchange2_name}")

    if not common_symbols:
        raise ValueError(f"❌ No common **USDT** trading pairs found between {exchange1_name} and {exchange2_name}.")

    # Connect to exchanges
    exchange1 = getattr(ccxt, exchange1_name)()
    exchange2 = getattr(ccxt, exchange2_name)()

    results = []

    for symbol in common_symbols:
        print(f"🔄 Fetching data for {symbol}...")

        try:
            # Fetch latest prices
            price1 = exchange1.fetch_ticker(symbol).get("last", None)
            price2 = exchange2.fetch_ticker(symbol).get("last", None)

            # Ensure prices are valid
            if price1 is None or price2 is None:
                print(f"⚠️ Missing price data for {symbol}, skipping...")
                continue

            # Calculate spread percentage
            spread = abs(price1 - price2) / ((price1 + price2) / 2) * 100

            # Get trading fees (Default to 0.1% per trade)
            fee1 = exchange1_info["Trading Fees"]
            fee2 = exchange2_info["Trading Fees"]

            # Fetch liquidity dynamically
            liquidity1 = get_liquidity(exchange1_name, symbol)
            liquidity2 = get_liquidity(exchange2_name, symbol)

            # Calculate potential profit
            potential_profit = calculate_potential_profit(price1, price2, fee1["taker_fee"], fee2["taker_fee"], TRADE_AMOUNT)

            results.append({
                "Symbol": symbol,
                "Exchange 1": exchange1_name,
                "Exchange 2": exchange2_name,
                "Price 1": price1,
                "Price 2": price2,
                "Average Price": (price1 + price2) / 2,
                "Spread (%)": spread,
                "Exchange 1 Maker Fee": fee1["maker_fee"],
                "Exchange 1 Taker Fee": fee1["taker_fee"],
                "Exchange 2 Maker Fee": fee2["maker_fee"],
                "Exchange 2 Taker Fee": fee2["taker_fee"],
                "Exchange 1 Liquidity (24h Vol)": liquidity1,
                "Exchange 2 Liquidity (24h Vol)": liquidity2,
                "Potential Profit ($)": potential_profit
            })

        except Exception as e:
            print(f"⚠️ Error fetching data for {symbol}: {e}")
            continue  # Skip symbols that cause errors

    # Convert to a DataFrame for better visualization
    df = pd.DataFrame(results)

    # Replace NaN values with "N/A"
    df.fillna("N/A", inplace=True)

    if df.empty:
        raise ValueError("❌ No valid trading pairs with price and liquidity data found.")

    return df

# --- Function to Calculate Potential Profit ---
def calculate_potential_profit(price1, price2, fee1, fee2, trade_amount):
    """Calculates the potential profit for a given trade amount."""
    
    # Determine Buy & Sell Prices
    buy_price = min(price1, price2)
    sell_price = max(price1, price2)

    # Determine Fees
    buy_fee = fee1 if buy_price == price1 else fee2
    sell_fee = fee2 if sell_price == price2 else fee1

    # Calculate Crypto Amount Bought
    crypto_amount = trade_amount / buy_price

    # Calculate Trading Fees
    buy_fee_cost = buy_price * crypto_amount * buy_fee
    sell_fee_cost = sell_price * crypto_amount * sell_fee
    total_fees = buy_fee_cost + sell_fee_cost

    # Calculate Raw Profit
    raw_profit = (sell_price - buy_price) * crypto_amount

    # Calculate Final Profit (After Fees)
    final_profit = raw_profit - total_fees
    return round(final_profit, 4)  # Round to 4 decimal places for clarity

# --- Run & Save Results ---
if not os.path.exists(RESULTS_FILE):
    try:
        exchange1 = "binance"
        exchange2 = "bybit"
        result_df = compare_fees_spreads_liquidity(exchange1, exchange2)

        # Save results
        result_df.to_csv(RESULTS_FILE, index=False)
        print(f"📁 Results saved to {RESULTS_FILE}")

        # Display the results
        display(result_df)

    except ValueError as e:
        print(e)
else:
    result_df = pd.read_csv(RESULTS_FILE)
    print("✅ Loaded saved results.")
    display(result_df)

